In [96]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from bs4 import BeautifulSoup
import requests
headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"}

In [97]:
# List of state abbreviations
states = ['AL', 'AK', 'AZ' ,'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN',
 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV',
 'NH', 'NJ', 'NM', 'NY', 'DC', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD',
 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']


In [98]:
list = []
x = 0
#df = pd.json_normalize(res.json())
for state in states:
    res = requests.get(f'https://chronicdata.cdc.gov/resource/eav7-hnsx.json?measureid=MHLTH&$limit=4000&stateabbr={state}')
    list.append(res.json())
    res = requests.get(f'https://chronicdata.cdc.gov/resource/eav7-hnsx.json?measureid=DEPRESSION&$limit=4000&stateabbr={state}')
    list.append(res.json())
    
    

In [99]:
df = pd.json_normalize(list[0])
df.set_index('locationid')
for i in range(1,len(list)):
    curr = pd.json_normalize(list[i])
    df = df.append(curr)


In [100]:
# Cleaning up DF a little bit

df = df.rename(columns={'statedesc': 'state', 'locationname': 'city', 'measure': 'symptom', 'data_value': 'percent'})
del df['datasource']
del df['measureid']
del df['geolocation.type']
del df['geolocation.coordinates']
del df['data_value_footnote_symbol']
del df['data_value_footnote']
del df['low_confidence_limit']
del df['high_confidence_limit']
del df['categoryid']
del df['datavaluetypeid']

In [101]:
df.loc[((df['stateabbr'] == 'MD') & (df['city'] == 'Clarksburg'))]
#df.loc[df['stateabbr'] == 'MD']['locationname'].unique()

,year,stateabbr,state,city,category,symptom,data_value_unit,data_value_type,percent,totalpopulation,locationid,short_question_text
255,2019,MD,Maryland,Clarksburg,Health Status,Mental health not good for >=14 days among adu...,%,Age-adjusted prevalence,9.4,13766,2417350,Mental Health
353,2019,MD,Maryland,Clarksburg,Health Status,Mental health not good for >=14 days among adu...,%,Crude prevalence,9.7,13766,2417350,Mental Health
36,2019,MD,Maryland,Clarksburg,Health Outcomes,Depression among adults aged >=18 years,%,Age-adjusted prevalence,13.9,13766,2417350,Depression
542,2019,MD,Maryland,Clarksburg,Health Outcomes,Depression among adults aged >=18 years,%,Crude prevalence,14.4,13766,2417350,Depression


In [102]:
# Import Income Data Set
income_df = pd.read_csv("kaggle_income.csv", encoding='latin-1')

income_df['loc_fullname'] = income_df['City'] + ', ' + income_df['State_Name']

# delete unnecessary columns
del income_df['id']
del income_df['Type']
del income_df['County']
del income_df['Place']
del income_df['Primary']
del income_df['Lat']
del income_df['Lon']
del income_df['sum_w']
del income_df['Stdev']
del income_df['Area_Code']
del income_df['ALand']
del income_df['AWater']

# combine duplicate cities
grouped = income_df.groupby(['loc_fullname'], as_index=False)
income_df = grouped.agg('mean')


income_df.loc[income_df['loc_fullname'] == 'Rockville, Maryland']

,loc_fullname,State_Code,Zip_Code,Mean,Median
8633,"Rockville, Maryland",24.0,20851.833333,116024.416667,185370.833333


In [103]:

income_df.loc[income_df['loc_fullname'] == 'Rockville, Maryland']

,loc_fullname,State_Code,Zip_Code,Mean,Median
8633,"Rockville, Maryland",24.0,20851.833333,116024.416667,185370.833333


In [104]:
# import us cities dataset
cities_df = pd.read_csv('uscities.csv')
del cities_df['city_ascii']
del cities_df['county_name']
del cities_df['lat']
del cities_df['lng']
del cities_df['source']
del cities_df['military']
del cities_df['incorporated']
del cities_df['timezone']
del cities_df['ranking']
del cities_df['id']

cities_df = cities_df.rename(columns={'population': 'Total Population'})
cities_df['loc_fullname'] = cities_df['city'] + ', ' + cities_df['state_name']

cities_df.head(10)

,city,state_id,state_name,county_fips,Total Population,density,zips,loc_fullname
0,New York,NY,New York,36061,18713220,10715,11229 11226 11225 11224 11222 11221 11220 1138...,"New York, New York"
1,Los Angeles,CA,California,6037,12750807,3276,90291 90293 90292 91316 91311 90037 90031 9000...,"Los Angeles, California"
2,Chicago,IL,Illinois,17031,8604203,4574,60018 60649 60641 60640 60643 60642 60645 6064...,"Chicago, Illinois"
3,Miami,FL,Florida,12086,6445545,5019,33129 33125 33126 33127 33128 33149 33144 3314...,"Miami, Florida"
4,Dallas,TX,Texas,48113,5743938,1526,75287 75098 75233 75254 75251 75252 75253 7503...,"Dallas, Texas"
5,Philadelphia,PA,Pennsylvania,42101,5649300,4554,19154 19151 19150 19153 19152 19102 19103 1910...,"Philadelphia, Pennsylvania"
6,Houston,TX,Texas,48201,5464251,1399,77069 77068 77061 77060 77063 77062 77065 7706...,"Houston, Texas"
7,Atlanta,GA,Georgia,13121,5449398,1441,30334 30331 30332 30309 30308 30305 30307 3030...,"Atlanta, Georgia"
8,Washington,DC,District of Columbia,11001,5379184,4457,20010 20011 20012 20015 20228 20520 20307 2041...,"Washington, District of Columbia"
9,Boston,MA,Massachusetts,25025,4688346,5532,02120 02121 02122 02124 02125 02126 02127 0212...,"Boston, Massachusetts"


In [105]:
# Merging all 3 dfs

df['loc_fullname'] = df['city'] + ', ' + df['state']
df = df.merge(income_df[['loc_fullname', 'Mean', 'Median', 'Zip_Code']], left_on='loc_fullname', right_on='loc_fullname')
df = df.merge(cities_df[['loc_fullname', 'Total Population']])
df.loc[df['loc_fullname'] == 'Rockville, Maryland']

,year,stateabbr,state,city,category,symptom,data_value_unit,data_value_type,percent,totalpopulation,locationid,short_question_text,loc_fullname,Mean,Median,Zip_Code,Total Population
14336,2019,MD,Maryland,Rockville,Health Status,Mental health not good for >=14 days among adu...,%,Crude prevalence,10.3,61308,2467675,Mental Health,"Rockville, Maryland",116024.416667,185370.833333,20851.833333,68079
14337,2019,MD,Maryland,Rockville,Health Status,Mental health not good for >=14 days among adu...,%,Age-adjusted prevalence,10.6,61308,2467675,Mental Health,"Rockville, Maryland",116024.416667,185370.833333,20851.833333,68079
14338,2019,MD,Maryland,Rockville,Health Outcomes,Depression among adults aged >=18 years,%,Crude prevalence,15.8,61308,2467675,Depression,"Rockville, Maryland",116024.416667,185370.833333,20851.833333,68079
14339,2019,MD,Maryland,Rockville,Health Outcomes,Depression among adults aged >=18 years,%,Age-adjusted prevalence,15.9,61308,2467675,Depression,"Rockville, Maryland",116024.416667,185370.833333,20851.833333,68079
